In [69]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
from functools import *
import itertools as it

In [25]:
def print_board(board):
    for row in board:
        print(*row)

# Frequency Matrix

In [3]:
def flatten_caps(word):
    ret = ''
    for c in word:
        if c.isupper():
            ret += 'A'
        ret += c.lower()
    return ret

In [44]:
def get_freq_matrix(dictionary, v=False):
    LETTERS = set()
    for word in dictionary:
        LETTERS = LETTERS.union(set(word))
    LETTERS = sorted(list(LETTERS))
    LETTERS_INDEX = dict([el[::-1] for el in enumerate(LETTERS)])
    
    num_letters = len(LETTERS)
    letter_frequency = np.zeros(num_letters)
    transition_frequency = np.zeros((num_letters, num_letters))

    for word in dictionary:
        for i in range(len(word) - 1):
            letter_frequency[LETTERS_INDEX[word[i]]] += 1
            transition_frequency[LETTERS_INDEX[word[i]]][LETTERS_INDEX[word[i + 1]]] += 1
            
    # print(sum(transition_frequency))
    # transition_frequency_normalized = np.array([row / sum(row) if sum(row) > 0 else row for row in transition_frequency])
    
    if v:
        plt.bar([el for el in LETTERS], letter_frequency);
        plt.matshow(transition_frequency_normalized);
        plt.axis('off')
        for i in range(len(LETTERS)):
            plt.text(i - 0.5, -1, LETTERS[i])
            plt.text(-1.5, i + 0.5, LETTERS[i])
            
    return transition_frequency / sum(sum(transition_frequency)), LETTERS

# Cost Function

In [87]:
@lru_cache(maxsize = None)
def get_distance_matrix(l, w):
    """
    given a length and width of grid, find the minimum distance between each two entries
    
    returns a matrix
    """
    
    ret = np.zeros((l * w, l * w))
    for i in range(l * w):
        for j in range(l * w):
            x1, y1 = divmod(i, w)
            x2, y2 = divmod(j, w)
            ret[i][j] = min(abs(x1 - x2), w - abs(x1 - x2)) + abs(y1 - y2)
            
    return ret

In [6]:
def flatten(board):
    ret = []
    for row in board:
        ret.extend(row)
    return ret

In [7]:
def get_perm_matrix(perm):
    ret = np.zeros((len(perm), len(perm)))
    for i in range(len(perm)):
        ret[i][perm[i]] += 1
    return ret

In [8]:
def get_conjugate_distance_matrix(board):
    """
    board is a grid of letters
    use `get_distance_matrix` to find the matrix of distances between letter pairs
    `get_distance_matrix` should be cached, so this should be faster
    
    in other words, return ret = `get_distance_matrix` with permuted rows and cols
    so that ret[i][j] is the distance between the ith and jth letters instead of the 
    [i // w][i % w] [j // w][j % w] positions
    
    This should be done by conjugating `get_distance_matrix` with the permutation
    matrix for the permutation flatten(board) with respect to sorted(flatten(board))

    """
    
    dist_mat = get_distance_matrix(len(board), len(board[0]))
    flattened_board = flatten(board)
    sorted_flattened_board = sorted(flattened_board)
    perm_matrix = get_perm_matrix([flattened_board.index(c) for c in sorted_flattened_board])
    
    return perm_matrix @ dist_mat @ np.linalg.inv(perm_matrix)

In [9]:
board = [
    ['b', 'f', 'd'],
    ['c', 'e', 'a']
]
print(get_distance_matrix(2, 3))
print()
print(get_conjugate_distance_matrix(board))

[[0. 1. 2. 1. 2. 3.]
 [1. 0. 1. 2. 1. 2.]
 [2. 1. 0. 3. 2. 1.]
 [1. 2. 3. 0. 1. 2.]
 [2. 1. 2. 1. 0. 1.]
 [3. 2. 1. 2. 1. 0.]]

[[0. 3. 2. 1. 1. 2.]
 [3. 0. 1. 2. 2. 1.]
 [2. 1. 0. 3. 1. 2.]
 [1. 2. 3. 0. 2. 1.]
 [1. 2. 1. 2. 0. 1.]
 [2. 1. 2. 1. 1. 0.]]


In [10]:
def cost(board, freq_matrix):
    """
    board should be rectangle
    freq_matrix gives frequency a matrix populated by a dictionary
    they should be the same dimension
    
    """
    return sum(sum(freq_matrix * get_conjugate_distance_matrix(board)))

# Optimization

In [45]:
dictionary = []
with open('words.txt') as f:
    for word in f.readlines():
        dictionary.append(flatten_caps(word.strip()))

In [88]:
freq_matrix, LETTERS = get_freq_matrix(dictionary)

In [47]:
print("".join(LETTERS))

!&',-./0123456789Aabcdefghijklmnopqrstuvwxyz


## Comparison

In [98]:
board = np.array([
    ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '!'],
    ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '&'],
    ['a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', "'", 'A'],
    ['z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '-']
])

In [99]:
cost(board, freq_matrix)

4.129801459773482

## Random

In [89]:
best_cost = 100 ** 100
best_board = None
for _ in range(10_000):
    letters = LETTERS.copy()
    random.shuffle(letters)
    board = np.array(letters).reshape((4, 11))
    c = cost(board, freq_matrix)
    if c < best_cost:
        best_board = board
        best_cost = c

In [90]:
print(best_cost)
print()
print_board(best_board)

3.3431752371765384

& m 7 o c v 1 0 8 ! /
' 2 d 9 i r s a - q x
6 y e A t h p n j w 4
5 f u l b k g 3 . , z


## Niave tree search

Start with best_board from above. Look at all the children by doing a swap. Transition to the child with lowest cost.

This is bad for several reasons, but first, is that it could be that a every child has greater cost. Then we are getting worse.

In [91]:
def get_children_by_swap(board):
    l = len(board)
    w = len(board[0])
    for i, j in it.permutations(range(l * w), r=2):
        child = board.copy()
        child[divmod(i, w)], child[divmod(j, w)] = child[divmod(j, w)], child[divmod(i, w)]
        yield child
    return None

In [93]:
board = best_board.copy()
for _ in range(10):
    board = sorted(
        ((cost(child, freq_matrix), child) for child in get_children_by_swap(board)),
        key = lambda x: x[0]
    )[0][1]

In [94]:
print_board(board)
print()
print(cost(board, freq_matrix))

& k g o c v z 0 8 ! /
' w p n i r b A 7 q x
6 y h a t e d 9 j 2 4
5 f u l s m - 3 . , 1

2.4844732890476235


## Old

In [ ]:
def random_swaps(permutation, num_swaps):
    for i in range(num_swaps):
        r1 = random.randint(0, 25)
        r2 = random.randint(0, 25)
        while r1 == r2:
            r2 = random.randint(0, 25)
        permutation[r1], permutation[r2] = permutation[r2], permutation[r1]
    return permutation

In [ ]:
def optimal_num_children(num_swaps):
    total_number_of_children = 350 ** num_swaps
    # percent of population for half confidence of seeing all children
    # https://www.desmos.com/calculator/d1svxeq9uj
    perc = np.math.log(0.7 * (total_number_of_children + 2), np.math.e) + 0.7
    return int(perc * total_number_of_children)

In [ ]:
best_permutation = None
new_best_permutation = list(np.random.permutation(26).data)
best_cost = cost(new_best_permutation)

for num_swaps in [10, 10, 9, 9, 8, 8, 7, 6, 5, 4, 4, 4, 4] + ([3] * 10) + ([2] * 20) + ([1] * 10):
    best_permutation = new_best_permutation.copy()
    for _ in range(100):
        new_permutation = random_swaps(best_permutation.copy(), num_swaps)
        c = cost(new_permutation)
        if c < best_cost:
            best_cost = c
            new_best_permutation = new_permutation.copy()
    print(num_swaps, '  best:', best_cost)

In [ ]:
print(new_best_permutation)
print_board(new_best_permutation)
cost(new_best_permutation)